In [ ]:
#@title ドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title モジュールのインストール
%%capture
# Nanostat
!pip install Nanostat
!pip install Nanostat --upgrade
# NanoPlot
!pip install NanoPlot
!pip install NanoPlot --upgrade
# Nanofilt
!pip install Nanofilt
!pip install Nanofilt --upgrade
# biopython
!pip install Biopython

In [ ]:
#@title モジュールをインポート
import Bio
from Bio import SeqIO
import numpy as np
import inspect
import nanostat
import nanoplot
import nanofilt
import matplotlib.pyplot as plt

In [ ]:
#@title nanoporeの出力をまとめる
#@markdown 以下の内容を入力して実行する<br><br>
#@markdown nanoporeの出力(fastq.gz)が入ったディレクトリ
input_dir="/content/drive/MyDrive/RNAseq/Inputs/\u5B9F\u9A13\u540D/\u30B5\u30F3\u30D7\u30EB\u540D"  #@param {type:"string"}
#@markdown 出力を纏めた後のファイル名(拡張子を含めない)
file_name="\u51FA\u529B\u3092\u7E8F\u3081\u305F\u5F8C\u306E\u30D5\u30A1\u30A4\u30EB\u540D\u3092\u6307\u5B9A" #@param {type:"string"}
#@markdown <br>出力を纏めたものは、入力ディレクトリの1つ上のディレクトリに保存される


# zcatでinput_dir内のfastq.gzファイルを纏めてfile_nameに出力
!zcat {input_dir}/barcode01/*.fastq.gz {input_dir}/barcode03_1/*.fastq.gz {input_dir}/barcode03_2/*.fastq.gz > {file_name}.fastq
# 入力ディレクトリに移動
%cd $input_dir
# 一つ上のディレクトリに保存
!cp /content/{file_name}.fastq ..
# もとのディレクトリに戻る
%cd /content

In [ ]:
%%capture
#@title 元データでnanoplotを利用してQC
%cd $input_dir/..
!NanoPlot --fastq {file_name}.fastq --loglength -t 12 -o {file_name}_raw_read_qc --drop_outliers 
!NanoStat --fastq {file_name}.fastq -n NanoStat.txt --outdir {file_name}_raw_read_qc/statreports
%cd /content

In [ ]:
%%capture
#@title nanofiltを利用してクオリティを調整
#@markdown リードクオリティが特に低い、前方50塩基をトリミング
!NanoFilt {file_name}.fastq \
          --headcrop 50 \
          > front_trimed_{file_name}.fastq

# 入力ディレクトリに移動
%cd $input_dir
# 一つ上のディレクトリに保存
!cp /content/front_trimed_{file_name}.fastq $input_dir/..
# もとのディレクトリに戻る
%cd /content


In [ ]:
%%capture
#@title 先頭をトリムしたデータで再度nanoplotを利用してQC
%cd $input_dir/..
!NanoPlot --fastq front_trimed_{file_name}.fastq --loglength -t 12 -o {file_name}_front_trimed_read_qc --drop_outliers 
!NanoStat --fastq front_trimed_{file_name}.fastq -n NanoStat.txt --outdir {file_name}_front_trimed_read_qc/statreports
%cd /content

In [ ]:
%%capture
#@title nanofiltを利用してクオリティを調整
#@markdown リードクオリティが一定以下のものを切り捨てる<br>前方50塩基を切り捨て
quality_level="10" #@param {type:"string"}
!NanoFilt front_trimed_{file_name}.fastq \
          --headcrop 50 \
          -q {quality_level} \
          > trimed_{file_name}.fastq

# 入力ディレクトリに移動
%cd $input_dir
# 一つ上のディレクトリに保存
!cp /content/trimed_{file_name}.fastq $input_dir/..
# もとのディレクトリに戻る
%cd /content


In [ ]:
%%capture
#@title 調整したデータで再度nanoplotを利用してQC
%cd $input_dir/..
!NanoPlot --fastq trimed_{file_name}.fastq --loglength -t 12 -o {file_name}_trimed_read_qc --drop_outliers 
!NanoStat --fastq trimed_{file_name}.fastq -n NanoStat.txt --outdir {file_name}_trimed_read_qc/statreports
%cd /content

＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝<br>

In [ ]:
#@title KallistoへのPermission deniedを防ぐためにアクセス権を与える
!chmod 755 /content/drive/MyDrive/RNA-seq/Tools/kallisto/kallisto

Kallistoのindexファイルを作成する<br>
時間がかかるので、変更がない限りは初回のみの実行でよい

In [ ]:
%%bash capture
#@title Kallistoのindexファイルを作成する
#@markdown 時間がかかるため、k-merを変更しない限りは1度実行したら次からは実行しなくて良い<br>


#@markdown k-merの値を入力
k_mer="31" #@param {type:"string"}



/content/drive/MyDrive/RNA-seq/Tools/kallisto/kallisto \
index \
-i drive/MyDrive/RNA-seq/SourceFiles/transcripts_index_kallisto \
drive/MyDrive/RNA-seq/SourceFiles/gencode.v38.transcripts.fa.gz \
-k $k_mer

＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝<br>
kallistoを実行し、TPMを算出する<br>
シングルエンドの場合、以下でデータの確認をまず行う

In [ ]:
#@title シングルエンド用 - データ確認
records = SeqIO.parse("trimed_" + file_name + ".fastq" , "fastq")
sequence_length_list = []  # リードごとの塩基数を格納するリスト
len_sum = 0
GC_count = 0  # GCの合計を格納する
ATGC_count = 0  # ATGCの合計を格納する
 
# リードごとに解析
for record in records:
    sequence_length_list.append(len(record.seq))
    len_sum = len_sum + len(record.seq)
    GC_count += (record.seq.count('G') + record.seq.count('C'))
    ATGC_count += (record.seq.count('A') + record.seq.count('T') + record.seq.count('G') + record.seq.count('C'))
 
print("リード数：" + str(len(sequence_length_list)))
print("リード長：" + str(min(sequence_length_list)) + '-' + str(max(sequence_length_list)))
print("GC含量：" + str(GC_count / ATGC_count * 100) + '%')
print("平均リード長(-l)：" +  str(len_sum / len(sequence_length_list)))
print("偏差(-s)：" + str(np.std(sequence_length_list)))

ave_len=int(len_sum / len(sequence_length_list))
stdev=int(np.std(sequence_length_list))

In [ ]:
#@title シングルエンド用 - Kallisto
out_dir="/content/drive/MyDrive/RNA-seq/Outputs/\u5B9F\u9A13\u540D/\u30B5\u30F3\u30D7\u30EB\u540D" #@param {type:"string"}

DataType="TrimedData" #@ param ["RawData", "TrimedData"]

if DataType == "RawData":
  !mkdir -p $out_dir
  !/content/drive/MyDrive/RNA-seq/Tools/kallisto/kallisto \
  quant \
  -i /content/drive/MyDrive/RNA-seq/SourceFiles/transcripts_index_kallisto \
  -o $out_dir \
  -t 2 \
  --single \
  -l "$ave_len" \
  -s "$stdev" \
  {file_name}.fastq
elif DataType =="TrimedData":
  !mkdir -p {out_dir}_trimed
  !/content/drive/MyDrive/RNA-seq/Tools/kallisto/kallisto \
  quant \
  -i /content/drive/MyDrive/RNA-seq/SourceFiles/transcripts_index_kallisto \
  -o {out_dir}_trimed \
  -t 2 \
  --single \
  -l "$ave_len" \
  -s "$stdev" \
  trimed_{file_name}.fastq